In [34]:
import numpy as np
import torch
import os 
import sys
from decision_transformer.evaluation.evaluate_episodes import evaluate_episode, evaluate_episode_rtg
from decision_transformer.models.decision_transformer import DecisionTransformer
from decision_transformer.models.mlp_bc import MLPBCModel
from decision_transformer.training.act_trainer import ActTrainer
from decision_transformer.training.seq_trainer import SequenceTrainer

PWD = os.getcwd()
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../../')
from util import *

class DTAgent: #TODO: Implement the agent here
    def __init__(self, model,
                    state_dim, action_dim,
                    state_mean=0., state_std=1.,
                    scale=1000.,
                    rtg=1000,
                    device='cuda',
                 ):
        self.model = model
        self.model.eval()
        self.mode = 'normal'
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.state_mean = torch.from_numpy(state_mean).to(device=device)
        self.state_std = torch.from_numpy(state_std).to(device=device)
        self.scale = scale
        self.device = device
        
        self.reset_rtg(rtg)
        
        
    def act(self, state, prev_rew=0 ):
        assert state.shape == (self.state_dim,)
        # action = np.random.uniform(-5, 5, size=(self.action_dim))
        if self.t==0:
            self.states = torch.from_numpy(state).reshape(1, self.state_dim).to(device=self.device, dtype=torch.float32)
            self.actions = torch.zeros((0, self.action_dim), device=self.device, dtype=torch.float32)
            self.rewards = torch.zeros(0, device=self.device, dtype=torch.float32)
        else:
            cur_state = torch.from_numpy(state).to(device=self.device).reshape(1, self.state_dim)
            states = torch.cat([states, cur_state], dim=0)
            self.rewards[-1] = prev_rew

            if self.mode != 'delayed':
                pred_return = target_return[0,-1] - (prev_rew/self.scale)
            else:
                pred_return = target_return[0,-1]
            target_return = torch.cat(
                [target_return, pred_return.reshape(1, 1)], dim=1)
            timesteps = torch.cat(
                [timesteps,
                torch.ones((1, 1), device=self.device, dtype=torch.long) * (self.t)], dim=1)
            
            
        self.actions = torch.cat([self.actions, torch.zeros((1, self.action_dim), device=self.device)], dim=0)
        self.rewards = torch.cat([self.rewards, torch.zeros(1, device=self.device)])
        
        action = self.model.get_action(
            (self.states.to(dtype=torch.float32) - self.state_mean) / self.state_std,
            self.actions.to(dtype=torch.float32),
            self.rewards.to(dtype=torch.float32),
            self.target_return.to(dtype=torch.float32),
            self.timesteps.to(dtype=torch.long),
        )
        self.actions[-1] = action
        action = action.detach().cpu().numpy()
        
        # state, reward, done, _, _ = env.step(action)
        

        self.t+=1
        
        return action
    
    def reset_rtg(self,rtg):
        self.target_return = torch.tensor(rtg, device=self.device, dtype=torch.float32).reshape(1, 1)
        self.timesteps = torch.tensor(0, device=self.device, dtype=torch.long).reshape(1, 1)
        self.t = 0
        
        ...
    
    

def eval_episodes(target_rew, num_eval_episodes, env_walk, env_run, state_dim, act_dim, max_ep_len, scale, state_mean, state_std, device, mode='normal'):
    def fn(model):
        returns_walk = []
        returns_run = []
        for _ in range(num_eval_episodes):
            with torch.no_grad():
                ret_walk, __ = evaluate_episode_rtg(
                    env_walk,
                    state_dim,
                    act_dim,
                    model,
                    max_ep_len=max_ep_len,
                    scale=scale,
                    target_return=target_rew/scale,
                    mode=mode,
                    state_mean=state_mean,
                    state_std=state_std,
                    device=device,
                )
                ret_run, __ = evaluate_episode_rtg(
                    env_run,
                    state_dim,
                    act_dim,
                    model,
                    max_ep_len=max_ep_len,
                    scale=scale,
                    target_return=target_rew/scale,
                    mode=mode,
                    state_mean=state_mean,
                    state_std=state_std,
                    device=device,
                )
                
            returns_walk.append(ret_walk)
            returns_run.append(ret_run)
        return {
            # f'target_{target_rew}_return_mean': np.mean(returns),
            # f'target_{target_rew}_return_std': np.std(returns),
            f'target_{target_rew}_return_walk_mean': np.mean(returns_walk),
            f'target_{target_rew}_return_walk_std': np.std(returns_walk),
            f'target_{target_rew}_return_walk_all': returns_walk, 
            f'target_{target_rew}_return_run_mean': np.mean(returns_run),
            f'target_{target_rew}_return_run_std': np.std(returns_run),
            f'target_{target_rew}_return_run_all': returns_run,
        }
    return fn


def evaluate_episode_rtg(
        env,
        state_dim,
        act_dim,
        model,
        max_ep_len=1000,
        scale=1000.,
        state_mean=0.,
        state_std=1.,
        device='cuda',
        target_return=None,
        mode='normal',
    ):

    model.eval()
    model.to(device=device)

    state_mean = torch.from_numpy(state_mean).to(device=device)
    state_std = torch.from_numpy(state_std).to(device=device)

    state, _ = env.reset()
    if mode == 'noise':
        state = state + np.random.normal(0, 0.1, size=state.shape)

    # we keep all the histories on the device
    # note that the latest action and reward will be "padding"
    states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
    actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)
    rewards = torch.zeros(0, device=device, dtype=torch.float32)

    ep_return = target_return
    target_return = torch.tensor(ep_return, device=device, dtype=torch.float32).reshape(1, 1)
    timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)

    sim_states = []

    episode_return, episode_length = 0, 0
    for t in range(max_ep_len):

        # add padding
        actions = torch.cat([actions, torch.zeros((1, act_dim), device=device)], dim=0)
        rewards = torch.cat([rewards, torch.zeros(1, device=device)])

        action = model.get_action(
            (states.to(dtype=torch.float32) - state_mean) / state_std,
            actions.to(dtype=torch.float32),
            rewards.to(dtype=torch.float32),
            target_return.to(dtype=torch.float32),
            timesteps.to(dtype=torch.long),
        )
        actions[-1] = action
        action = action.detach().cpu().numpy()

        state, reward, done, _, _ = env.step(action)
        # print('step done!!!')

        cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
        states = torch.cat([states, cur_state], dim=0)
        rewards[-1] = reward

        if mode != 'delayed':
            pred_return = target_return[0,-1] - (reward/scale)
        else:
            pred_return = target_return[0,-1]
        target_return = torch.cat(
            [target_return, pred_return.reshape(1, 1)], dim=1)
        timesteps = torch.cat(
            [timesteps,
             torch.ones((1, 1), device=device, dtype=torch.long) * (t+1)], dim=1)

        episode_return += reward
        episode_length += 1

        if done:
            break

    return episode_return, episode_length



    

In [37]:

task_bit = 1

env_walk = get_gym_env('walk',seed=1, ADD_TASKBIT=task_bit)
env_run = get_gym_env('run',seed=1, ADD_TASKBIT=task_bit)
max_ep_len = 1000
env_targets = [0.8,1,1000, 800, 500, 300, 200]  # evaluation conditioning targets
scale = 1000.  # normalization for rewards/returns

state_dim = env_walk.observation_space.shape[0]
act_dim = env_walk.action_space.shape[0]

mode = 'normal'
K= 20
variant={
    'embed_dim': 256,
    'n_layer': 3,
    'n_head': 1,
    'activation_function': 'relu',
    'dropout': 0.1,
    'USE_DATASET_STR': '__all__',
    'task_bit':task_bit,
}

    

In [30]:


dt_model = DecisionTransformer(
            state_dim=state_dim,
            act_dim=act_dim,
            max_length=K,
            max_ep_len=max_ep_len,
            hidden_size=variant['embed_dim'],
            n_layer=variant['n_layer'],
            n_head=variant['n_head'],
            n_inner=4*variant['embed_dim'],
            activation_function=variant['activation_function'],
            n_positions=1024,
            resid_pdrop=variant['dropout'],
            attn_pdrop=variant['dropout'],
        )
dt_model.load("/home/wjxie/wjxie/env/offline_multitask/ckpt/_dt/9.pt")

In [13]:
print(os.getcwd())

/NAS2020/Workspaces/DRLGroup/wjxie/env/offline_multitask/decision-transformer/gym


In [26]:
from experiment_dmc import from_datasetstr_to_datasetfilepath, make_trajs, read_data
dataset_file_paths = from_datasetstr_to_datasetfilepath(variant['USE_DATASET_STR'])
trajectories = make_trajs(dataset_file_paths)
states, traj_lens, returns = read_data(trajectories,mode,variant['task_bit'])
traj_lens, returns = np.array(traj_lens), np.array(returns)
print('########### Data Loaded! ###########')

# used for input normalization
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
print('state_mean:', state_mean)
print('state_std:', state_std)

100%|██████████| 204/204 [00:00<00:00, 367.21it/s]


########### Data Loaded! ###########
state_mean: [ 9.31270788e-01 -6.56689485e-02  7.83823628e-01 -3.48372718e-01
  5.37133719e-01  6.69629078e-01  4.76454052e-01  4.68625933e-01
  8.82615255e-01  7.20799534e-02  5.24125645e-01  7.15013248e-01
  4.73387288e-01  2.81045324e-01  1.09207265e+00 -9.45097368e-03
  1.59500805e+00 -1.04131104e-03 -1.72432532e-02  1.78307676e-02
  3.09820980e-02 -3.02996599e-02  1.19112037e-02  5.32888793e-03
  5.00000000e-01]
state_std: [0.22458547 0.27925738 0.29520055 0.42085037 0.40492277 0.31483846
 0.59775127 0.44280557 0.26616778 0.38072404 0.38143853 0.26182876
 0.69636542 0.46042971 0.20927246 1.59334934 0.62619761 4.48222262
 9.71117892 6.18426965 8.2479309  9.39151823 6.45248379 7.34752834
 0.500001  ]


In [38]:
from pprint import pprint
for target_return in env_targets:
    pprint(eval_episodes(target_return, 20, env_walk, env_run, state_dim, act_dim, max_ep_len, scale, state_mean, state_std, 'cuda', mode=mode)(dt_model))

/NAS2020/Workspaces/DRLGroup/wjxie/anaconda3/envs/rlp2/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
